In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    # "FASTX",
    # "StatsBase",
    # "Distributions",
    # "StatsPlots",
    # "Random",
    # "Dates",
    # "DataFrames",
    # "BioSequences",
    # "Conda",
    # "Downloads"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
bams = sort(filter(x -> occursin(r"\.sorted\.bam$", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
bam = first(bams)
fasta_reference = replace(bam, r"\.normalized\.vcf\.fna.*" => "")
outfile = bam * ".gatk.vcf"

In [ ]:
Pkg.build("Mycelia")

In [ ]:
fasta_dict = replace(fasta_reference, ".fna" => ".dict")

In [ ]:
# run(`$(Mycelia.MAMBA) run --live-stream -n picard java -jar picard.jar CreateSequenceDictionary R=$(fasta_reference) O=$(fasta_dict)`)

In [ ]:
# run(`$(Mycelia.MAMBA) run --live-stream -n gatk4 gatk --java-options "-Xmx8G" HaplotypeCaller --sample-ploidy 1 --reference $(fasta_reference) --input $(bam) --output $(outfile)`)

In [ ]:
# threads = 1
# for bam in bams[2:end]
#     outfile = bam * ".bcftools.vcf"
#     fasta_reference = replace(bam, r"\.normalized\.vcf\.fna.*" => "")
#     if !isfile(outfile) || (filesize(outfile) == 0)
#         @info "$(outfile) not present, generating..."        
#         cmd = 
#         """
#         $(Mycelia.MAMBA) run --live-stream -n gatk gatk 
#         $(Mycelia.MAMBA) run --live-stream -n bcftools bcftools mpileup --threads $(threads) --output-type u --fasta-ref $(fasta_reference) $(bam) \\
#         | $(Mycelia.MAMBA) run --live-stream -n bcftools bcftools call --threads $(threads) --multiallelic-caller --variants-only --output-type v --output $(outfile)
#         """
#         Mycelia.nersc_sbatch(
#             job_name = "$(basename(outfile))",
#             mail_user = "cameron.prybol@gmail.com",
#             logdir = mkpath("$(homedir())/workspace/slurmlogs"),
#             qos = "shared",
#             cpus_per_task= threads,
#             cmd = cmd)
#     else
#         @info "$(outfile) already present..."
#     end
# end